# Movie Recommendation System (1M)

## Initial library load 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
writer = SummaryWriter()  

## Data load

In [18]:
ratings = pd.read_csv("./ml-1m/ratings.dat", sep="::", header=None,
                      names=['UserID','MovieID','Rating','Timestamp'], engine="python")

In [20]:
movies = pd.read_csv("./ml-1m/movies.dat", sep="::", header=None,
                     names=['MovieID','Title','Genre'], engine="python",
                    encoding="utf-8",
                    lineterminator="\n",
                    on_bad_lines="skip")

In [24]:
movies.head(20)

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [19]:
users = pd.read_csv("./ml-1m/users.dat", sep="::", engine="python",
                    header=None, names=['UserID','Gender','Age','Occupation','Zip-code'])

In [28]:
users.head(10)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [10]:
ratings.head(10)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [26]:
movies_ratings = pd.merge(ratings,movies, on="MovieID", how="outer")

In [27]:
movies_ratings.head(10)

,UserID,MovieID,Rating,Timestamp,Title,Genre
0,1.0,1,5.0,978824268.0,Toy Story (1995),Animation|Children's|Comedy
1,6.0,1,4.0,978237008.0,Toy Story (1995),Animation|Children's|Comedy
2,8.0,1,4.0,978233496.0,Toy Story (1995),Animation|Children's|Comedy
3,9.0,1,5.0,978225952.0,Toy Story (1995),Animation|Children's|Comedy
4,10.0,1,5.0,978226474.0,Toy Story (1995),Animation|Children's|Comedy
5,18.0,1,4.0,978154768.0,Toy Story (1995),Animation|Children's|Comedy
6,19.0,1,5.0,978555994.0,Toy Story (1995),Animation|Children's|Comedy
7,21.0,1,3.0,978139347.0,Toy Story (1995),Animation|Children's|Comedy
8,23.0,1,4.0,978463614.0,Toy Story (1995),Animation|Children's|Comedy
9,26.0,1,3.0,978130703.0,Toy Story (1995),Animation|Children's|Comedy


In [29]:
complete_df = pd.merge(movies_ratings, users, on="UserID", how="outer")

In [30]:
complete_df.head(10)

,UserID,MovieID,Rating,Timestamp,Title,Genre,Gender,Age,Occupation,Zip-code
0,1.0,1,5.0,978824268.0,Toy Story (1995),Animation|Children's|Comedy,F,1.0,10.0,48067
1,1.0,48,5.0,978824351.0,Pocahontas (1995),Animation|Children's|Musical|Romance,F,1.0,10.0,48067
2,1.0,150,5.0,978301777.0,Apollo 13 (1995),Drama,F,1.0,10.0,48067
3,1.0,260,4.0,978300760.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,F,1.0,10.0,48067
4,1.0,527,5.0,978824195.0,Schindler's List (1993),Drama|War,F,1.0,10.0,48067
5,1.0,531,4.0,978302149.0,"Secret Garden, The (1993)",Children's|Drama,F,1.0,10.0,48067
6,1.0,588,4.0,978824268.0,Aladdin (1992),Animation|Children's|Comedy|Musical,F,1.0,10.0,48067
7,1.0,594,4.0,978302268.0,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,F,1.0,10.0,48067
8,1.0,595,5.0,978824268.0,Beauty and the Beast (1991),Animation|Children's|Musical,F,1.0,10.0,48067
9,1.0,608,4.0,978301398.0,Fargo (1996),Crime|Drama|Thriller,F,1.0,10.0,48067
